# For loops with cross apply

#### Step 1: Create instrument upsert view

```sql
-- ============================================================
-- Description:
-- In this file, we create view which uploads instruments
-- from a CSV file based on an EffectiveDate
-- ============================================================

@v =
use Sys.Admin.SetupView
--provider=TestInstrumentWriter
--parameters
FileName,Text,/luminesce-examples/instrument-test.csv,false
EffectiveFrom,DateTime,2020-01-01,false
-----------------

--Can't pass table parameters to the View, so for now have to read the file each time from Drive
--Could have a separate query to chunk the files
@@file = select #PARAMETERVALUE(FileName);

@data = use Drive.Csv with @@file
--file={@@file}
enduse;

@@dt = select datetime(#PARAMETERVALUE(EffectiveFrom));

@toWrite = select * from @data where EffectiveFrom = @@dt;

select * from Lusid.Instrument.Writer where ToWrite = @toWrite and EffectiveFrom = @@dt;

enduse;

select * from @v
```

#### Step 2: Loop over csv file

```sql
--============================================================
-- Description:
-- In this query, we loop over rows in a CSV file by date,
-- and load the batches to an instruments writer view
-- This is an example of how to implement a for loop
-- in Luminesce. For more details, see this page:
-- https://support.lusid.com/knowledgebase/article/KA-01692/en-us
--============================================================

@@file = select '/luminesce-examples/instrument-test.csv';

@data = use Drive.Csv with @@file
--file={@@file}
enduse;

@ef = select distinct(EffectiveFrom) as Value from @data;

select
    ef.Value,
    r.LusidInstrumentId,
    r.WriteErrorCode
from
    @ef ef
    cross apply
    (
        select * from TestInstrumentWriter w where w.FileName = @@file and w.EffectiveFrom = ef.Value
    ) r
```